In [1]:
import os

if(os.path.isfile('data/notMNIST.pickle')):
    print('file exists')
else:
    print('file doesn\'t exists')

file exists


In [3]:
import pickle

with open('data/notMNIST.pickle','rb') as f:
    pickle_data = pickle.load(f)

pickle_data.keys()

dict_keys(['valid_dataset', 'train_dataset', 'train_labels', 'test_labels', 'test_dataset', 'valid_labels'])

In [3]:
train_dataset = pickle_data['train_dataset']
test_labels = pickle_data['test_labels']
valid_labels = pickle_data['valid_labels']
train_labels = pickle_data['train_labels']
valid_dataset = pickle_data['valid_dataset']
test_dataset = pickle_data['test_dataset']
del pickle_data # save memory

In [26]:
import tensorflow as tf

epochs = 30
learning_rate = 0.01
batch_size=128

features_size = train_dataset.shape[1]
labels_size = train_labels.shape[1]

features = tf.placeholder(tf.float32,[None,features_size])
labels = tf.placeholder(tf.float32,[None,labels_size])


#  3 layer deep neural network
layer1_hidden_units = 300
layer2_hidden_units = 100
output_layer = labels_size

layer1_weights = tf.Variable(tf.truncated_normal([features_size,layer1_hidden_units]))
layer1_bias = tf.Variable(tf.truncated_normal([layer1_hidden_units]))

layer2_weights = tf.Variable(tf.truncated_normal([layer1_hidden_units,layer2_hidden_units]))
layer2_bias = tf.Variable(tf.truncated_normal([layer2_hidden_units]))

layer3_weights = tf.Variable(tf.truncated_normal([layer2_hidden_units,output_layer]))
layer3_bias = tf.Variable(tf.zeros([output_layer]))


# forward prop with relu activations

layer1_output = tf.nn.relu(tf.matmul(features,layer1_weights) + layer1_bias)
layer2_output = tf.nn.relu(tf.matmul(layer1_output,layer2_weights) + layer2_bias)
logits = tf.matmul(layer2_output,layer3_weights) + layer3_bias


reg_lamda = 0.01
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits,labels=labels)
                         + reg_lamda * tf.nn.l2_loss(layer1_weights)
                         + reg_lamda * tf.nn.l2_loss(layer1_bias)
                         + reg_lamda * tf.nn.l2_loss(layer2_weights)
                         + reg_lamda * tf.nn.l2_loss(layer2_bias)
                         + reg_lamda * tf.nn.l2_loss(layer3_weights)
                         + reg_lamda * tf.nn.l2_loss(layer3_bias))

optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)

accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(logits,1),tf.argmax(labels,1)),dtype=tf.float32)) * 100

init = tf.global_variables_initializer()

def batch_entries(features,labels):
    batches = []
    feature_size=features.shape[0]
    
    start_index=0
    while(start_index != feature_size):
        end_index = min(start_index + batch_size, feature_size)
        batches.append([features[start_index:end_index,:] ,labels[start_index:end_index,:]])
        start_index = end_index
    
    return batches



In [27]:
valid_feed = {features: valid_dataset , labels : valid_labels}
test_feed = {features: test_dataset , labels : test_labels}
with tf.Session() as sess:
    sess.run(init)
    train_acc = 0.0
    valid_acc =0.0
    Test_acc = 0.0
    for i in range(epochs):
        for data in batch_entries(train_dataset,train_labels):
            opt,l,train_acc = sess.run([optimizer,loss,accuracy],feed_dict={features: data[0] , labels : data[1]})
            #print('Train Accuracy after epoch: ',i,'loss is : ',l)
        valid_acc = sess.run(accuracy,feed_dict=valid_feed)
        print('Validation Accuracy after epoch: ',i,' is : ',valid_acc)
    Test_acc = sess.run(accuracy,feed_dict=test_feed) 
    print('Test Accuracy is : ',Test_acc)
    

Validation Accuracy after epoch:  0  is :  23.4533
Validation Accuracy after epoch:  1  is :  36.3467
Validation Accuracy after epoch:  2  is :  39.8
Validation Accuracy after epoch:  3  is :  44.8267
Validation Accuracy after epoch:  4  is :  51.9867
Validation Accuracy after epoch:  5  is :  59.1733
Validation Accuracy after epoch:  6  is :  66.5867
Validation Accuracy after epoch:  7  is :  69.7333
Validation Accuracy after epoch:  8  is :  72.88
Validation Accuracy after epoch:  9  is :  76.36
Validation Accuracy after epoch:  10  is :  77.68
Validation Accuracy after epoch:  11  is :  78.68
Validation Accuracy after epoch:  12  is :  79.4933
Validation Accuracy after epoch:  13  is :  80.16
Validation Accuracy after epoch:  14  is :  80.6267
Validation Accuracy after epoch:  15  is :  81.32
Validation Accuracy after epoch:  16  is :  81.9467
Validation Accuracy after epoch:  17  is :  82.4
Validation Accuracy after epoch:  18  is :  82.8
Validation Accuracy after epoch:  19  is : 